# Sistem Pendukung Keputusan: Prioritas Pengadaan Buku Perpustakaan

## Menggunakan Metode TOPSIS (Technique for Order Preference by Similarity to Ideal Solution)

**Tujuan:** Menentukan 5 buku yang paling diprioritaskan untuk diadakan/ditambah stoknya berdasarkan multiple criteria decision making.

**Dataset:** 
- Data Buku Perpustakaan.csv
- Stok Buku Penerbit.csv  
- Usulan Buku.csv

**Metode:** TOPSIS (Multi-Attribute Decision Making)

## 1. Import Required Libraries

In [ ]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

print("Library berhasil diimport!")
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)

## 2. Load and Explore Dataset

In [ ]:
# Load dataset dari 3 file CSV
try:
    # Load data buku perpustakaan
    df_perpus = pd.read_csv('Data Buku Perpustakaan.csv')
    print("✓ Data Buku Perpustakaan berhasil dimuat")
    print(f"Shape: {df_perpus.shape}")
    
    # Load data stok buku penerbit  
    df_penerbit = pd.read_csv('Stok Buku Penerbit.csv')
    print("✓ Stok Buku Penerbit berhasil dimuat")
    print(f"Shape: {df_penerbit.shape}")
    
    # Load data usulan buku
    df_usulan = pd.read_csv('Usulan Buku.csv')
    print("✓ Usulan Buku berhasil dimuat")
    print(f"Shape: {df_usulan.shape}")
    
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan - {e}")
    print("Pastikan file CSV berada di direktori yang sama dengan notebook ini")

In [ ]:
# Eksplorasi struktur data perpustakaan
print("=== DATA BUKU PERPUSTAKAAN ===")
print("\nKolom:", df_perpus.columns.tolist())
print("\nInfo dataset:")
df_perpus.info()
print("\nSample data:")
df_perpus.head()

In [ ]:
# Eksplorasi struktur data penerbit
print("=== DATA STOK BUKU PENERBIT ===")
print("\nKolom:", df_penerbit.columns.tolist())
print("\nInfo dataset:")
df_penerbit.info()
print("\nSample data:")
df_penerbit.head()

In [ ]:
# Eksplorasi struktur data usulan
print("=== DATA USULAN BUKU ===")
print("\nKolom:", df_usulan.columns.tolist())
print("\nInfo dataset:")
df_usulan.info()
print("\nSample data:")
df_usulan.head()

## 3. Data Preprocessing and Merging

In [ ]:
# Preprocessing: Standardisasi kolom ISBN
print("Preprocessing data...")

# Konversi ISBN ke string dan bersihkan
df_perpus['ISBN'] = df_perpus['ISBN'].astype(str).str.strip()
df_penerbit['ISBN'] = df_penerbit['ISBN'].astype(str).str.strip() 
df_usulan['ISBN'] = df_usulan['ISBN'].astype(str).str.strip()

# Cek duplikasi berdasarkan judul dan ISBN
print("\n=== DUPLIKASI CHECK ===")
print(f"Duplikasi di data perpustakaan (judul): {df_perpus['JUDUL'].duplicated().sum()}")
print(f"Duplikasi di data penerbit (judul): {df_penerbit['JUDUL'].duplicated().sum()}")
print(f"Duplikasi di data usulan (judul): {df_usulan['JUDUL'].duplicated().sum()}")

print(f"Duplikasi di data perpustakaan (ISBN): {df_perpus['ISBN'].duplicated().sum()}")
print(f"Duplikasi di data penerbit (ISBN): {df_penerbit['ISBN'].duplicated().sum()}")
print(f"Duplikasi di data usulan (ISBN): {df_usulan['ISBN'].duplicated().sum()}")

In [ ]:
# Menggabungkan data menggunakan pandas merge
# Pertama, gabungkan data usulan dengan penerbit berdasarkan ISBN
print("=== MERGING DATA ===")

# Step 1: Merge usulan dengan penerbit berdasarkan ISBN
df_merge_1 = pd.merge(df_usulan, df_penerbit, on='ISBN', how='inner', suffixes=('_usulan', '_penerbit'))
print(f"Hasil merge usulan + penerbit: {df_merge_1.shape[0]} buku")

# Step 2: Coba merge dengan data perpustakaan berdasarkan ISBN
df_final = pd.merge(df_merge_1, df_perpus, on='ISBN', how='outer', suffixes=('', '_perpus'))
print(f"Hasil merge final (usulan + penerbit + perpustakaan): {df_final.shape[0]} buku")

# Jika ada konflik kolom, pilih yang lebih relevan
# Karena ada beberapa kolom dengan nama sama, kita perlu handle dengan hati-hati
print("\nKolom hasil merge:")
print(df_final.columns.tolist())

In [ ]:
# Alternatif: Merge berdasarkan judul jika ISBN tidak cocok
print("=== ALTERNATIVE MERGE BERDASARKAN JUDUL ===")

# Standardisasi judul (hapus spasi ekstra, convert ke lowercase untuk matching)
df_usulan['JUDUL_CLEAN'] = df_usulan['JUDUL'].str.strip().str.lower()
df_penerbit['JUDUL_CLEAN'] = df_penerbit['JUDUL'].str.strip().str.lower()  
df_perpus['JUDUL_CLEAN'] = df_perpus['JUDUL'].str.strip().str.lower()

# Merge berdasarkan judul yang sudah dibersihkan
df_merge_judul = pd.merge(df_usulan, df_penerbit, on='JUDUL_CLEAN', how='inner', suffixes=('_usulan', '_penerbit'))
print(f"Hasil merge usulan + penerbit (berdasarkan judul): {df_merge_judul.shape[0]} buku")

# Cek sample hasil merge
print("\nSample hasil merge:")
df_merge_judul[['JUDUL_usulan', 'JUDUL_penerbit', 'JUMLAH USULAN', 'STOK', 'HARGA']].head()

## 4. Data Cleaning and Preparation

In [ ]:
# Bersihkan dan siapkan data untuk analisis TOPSIS
print("=== DATA CLEANING ===")

# Gunakan hasil merge berdasarkan judul
df_clean = df_merge_judul.copy()

# Bersihkan data yang akan digunakan sebagai kriteria
# Konversi HARGA ke numeric (handle jika ada format aneh)
df_clean['HARGA'] = pd.to_numeric(df_clean['HARGA'], errors='coerce')

# Handle missing values
print("Missing values sebelum cleaning:")
print(df_clean.isnull().sum())

# Drop rows dengan missing values di kolom penting
df_clean = df_clean.dropna(subset=['JUMLAH USULAN', 'STOK', 'HARGA'])

# Remove duplicates berdasarkan judul
df_clean = df_clean.drop_duplicates(subset=['JUDUL_CLEAN'])

# Reset index
df_clean = df_clean.reset_index(drop=True)

print(f"\nData setelah cleaning: {df_clean.shape[0]} buku")
print("Missing values setelah cleaning:")
print(df_clean.isnull().sum())

In [ ]:
# Tambahkan kriteria turunan dan persiapan data untuk TOPSIS
print("=== PERSIAPAN KRITERIA ===")

# Buat kriteria baru berdasarkan tahun terbit (semakin baru semakin baik)
df_clean['TAHUN TERBIT_usulan'] = pd.to_numeric(df_clean['TAHUN TERBIT_usulan'], errors='coerce')
df_clean['KEBARUAN'] = df_clean['TAHUN TERBIT_usulan']  # Benefit criterion

# Filter data yang akan digunakan untuk TOPSIS (minimal harus ada semua kriteria)
df_clean = df_clean.dropna(subset=['JUMLAH USULAN', 'STOK', 'HARGA', 'KEBARUAN'])

# Buat dataset final untuk TOPSIS
columns_needed = ['JUDUL_usulan', 'ISBN_usulan', 'JUMLAH USULAN', 'STOK', 'HARGA', 'KEBARUAN']
df_topsis = df_clean[columns_needed].copy()

# Rename kolom untuk clarity
df_topsis.columns = ['JUDUL', 'ISBN', 'JUMLAH_USULAN', 'STOK_PENERBIT', 'HARGA', 'KEBARUAN']

print(f"Data siap untuk TOPSIS: {df_topsis.shape[0]} buku")
print("\nSample data:")
df_topsis.head(10)

## 5. Define Criteria and Weights

In [ ]:
# Definisikan kriteria dan bobot untuk TOPSIS
print("=== DEFINISI KRITERIA DAN BOBOT ===")

# Kriteria yang akan digunakan (minimal 3 kriteria)
criteria = {
    'JUMLAH_USULAN': {
        'type': 'benefit',  # Semakin banyak usulan, semakin prioritas
        'weight': 0.35,
        'description': 'Jumlah usulan dari user (benefit - semakin banyak semakin baik)'
    },
    'STOK_PENERBIT': {
        'type': 'benefit',  # Semakin banyak stok tersedia, semakin baik
        'weight': 0.25, 
        'description': 'Ketersediaan stok di penerbit (benefit - semakin banyak semakin baik)'
    },
    'HARGA': {
        'type': 'cost',     # Semakin rendah harga, semakin baik
        'weight': 0.25,
        'description': 'Harga buku (cost - semakin murah semakin baik)'
    },
    'KEBARUAN': {
        'type': 'benefit',  # Semakin baru tahun terbit, semakin baik
        'weight': 0.15,
        'description': 'Tahun terbit buku (benefit - semakin baru semakin baik)'
    }
}

# Validasi bobot (harus total = 1)
total_weight = sum([criteria[k]['weight'] for k in criteria.keys()])
print(f"Total bobot: {total_weight}")

if abs(total_weight - 1.0) > 0.001:
    print("WARNING: Total bobot tidak sama dengan 1!")
else:
    print("✓ Total bobot valid")

# Tampilkan kriteria
print("\n=== KRITERIA YANG DIGUNAKAN ===")
for i, (criterion, details) in enumerate(criteria.items(), 1):
    print(f"{i}. {criterion}")
    print(f"   - Tipe: {details['type'].upper()}")
    print(f"   - Bobot: {details['weight']}")
    print(f"   - Deskripsi: {details['description']}")
    print()

## 6. Implement TOPSIS Method

In [ ]:
# Implementasi fungsi TOPSIS
class TOPSIS:
    def __init__(self, decision_matrix, weights, criteria_types):
        """
        decision_matrix: numpy array dengan baris = alternatif, kolom = kriteria
        weights: list bobot untuk setiap kriteria
        criteria_types: list dengan 'benefit' atau 'cost' untuk setiap kriteria
        """
        self.decision_matrix = np.array(decision_matrix)
        self.weights = np.array(weights)
        self.criteria_types = criteria_types
        self.normalized_matrix = None
        self.weighted_matrix = None
        self.ideal_best = None
        self.ideal_worst = None
        self.distances_best = None
        self.distances_worst = None
        self.scores = None
        
    def normalize_matrix(self):
        """Normalisasi matriks menggunakan vector normalization"""
        # Untuk setiap kolom, bagi dengan akar kuadrat dari jumlah kuadrat semua elemen
        squared_sums = np.sqrt(np.sum(self.decision_matrix ** 2, axis=0))
        self.normalized_matrix = self.decision_matrix / squared_sums
        return self.normalized_matrix
    
    def create_weighted_matrix(self):
        """Buat weighted normalized matrix"""
        if self.normalized_matrix is None:
            self.normalize_matrix()
        self.weighted_matrix = self.normalized_matrix * self.weights
        return self.weighted_matrix
    
    def determine_ideal_solutions(self):
        """Tentukan ideal positive dan negative solution"""
        if self.weighted_matrix is None:
            self.create_weighted_matrix()
            
        self.ideal_best = np.zeros(len(self.criteria_types))
        self.ideal_worst = np.zeros(len(self.criteria_types))
        
        for i, criterion_type in enumerate(self.criteria_types):
            if criterion_type == 'benefit':
                self.ideal_best[i] = np.max(self.weighted_matrix[:, i])
                self.ideal_worst[i] = np.min(self.weighted_matrix[:, i])
            else:  # cost
                self.ideal_best[i] = np.min(self.weighted_matrix[:, i])
                self.ideal_worst[i] = np.max(self.weighted_matrix[:, i])
                
        return self.ideal_best, self.ideal_worst
    
    def calculate_distances(self):
        """Hitung jarak ke ideal positive dan negative solution"""
        if self.ideal_best is None:
            self.determine_ideal_solutions()
            
        # Jarak euclidean ke ideal best dan worst
        self.distances_best = np.sqrt(np.sum((self.weighted_matrix - self.ideal_best) ** 2, axis=1))
        self.distances_worst = np.sqrt(np.sum((self.weighted_matrix - self.ideal_worst) ** 2, axis=1))
        
        return self.distances_best, self.distances_worst
    
    def calculate_scores(self):
        """Hitung skor TOPSIS (closeness coefficient)"""
        if self.distances_best is None:
            self.calculate_distances()
            
        # Closeness coefficient = d- / (d+ + d-)
        self.scores = self.distances_worst / (self.distances_best + self.distances_worst)
        return self.scores
    
    def run_topsis(self):
        """Jalankan seluruh proses TOPSIS"""
        self.normalize_matrix()
        self.create_weighted_matrix()
        self.determine_ideal_solutions()
        self.calculate_distances()
        self.calculate_scores()
        return self.scores

print("✓ Class TOPSIS berhasil didefinisikan")

## 7. Prepare Data for TOPSIS

In [ ]:
# Persiapkan data untuk algoritma TOPSIS
print("=== PERSIAPAN DATA UNTUK TOPSIS ===")

# Ambil kolom kriteria yang akan digunakan
criteria_columns = ['JUMLAH_USULAN', 'STOK_PENERBIT', 'HARGA', 'KEBARUAN']

# Buat decision matrix (matriks keputusan)
decision_matrix = df_topsis[criteria_columns].values

# Siapkan weights dan criteria types
weights = [criteria[col]['weight'] for col in criteria_columns]
criteria_types = [criteria[col]['type'] for col in criteria_columns]

print(f"Decision matrix shape: {decision_matrix.shape}")
print(f"Jumlah alternatif (buku): {decision_matrix.shape[0]}")
print(f"Jumlah kriteria: {decision_matrix.shape[1]}")

print("\nKriteria dan bobotnya:")
for i, col in enumerate(criteria_columns):
    print(f"{i+1}. {col}: {weights[i]} ({criteria_types[i]})")

print("\nSample decision matrix:")
sample_df = pd.DataFrame(decision_matrix[:5], 
                        columns=criteria_columns,
                        index=df_topsis['JUDUL'].iloc[:5])
print(sample_df)

In [ ]:
# Persiapkan data untuk algoritma TOPSIS
print("=== PERSIAPAN DATA UNTUK TOPSIS ===")

# Ekstrak matriks keputusan (hanya kolom kriteria)
criteria_columns = ['JUMLAH_USULAN', 'STOK_PENERBIT', 'HARGA', 'KEBARUAN']
decision_matrix = df_topsis[criteria_columns].values

# Ekstrak bobot dan tipe kriteria
weights = [criteria[col]['weight'] for col in criteria_columns]
criteria_types = [criteria[col]['type'] for col in criteria_columns]

print("Matriks keputusan shape:", decision_matrix.shape)
print("Jumlah alternatif (buku):", decision_matrix.shape[0])
print("Jumlah kriteria:", decision_matrix.shape[1])

print("\nBobot kriteria:", weights)
print("Tipe kriteria:", criteria_types)

# Cek ada missing values atau infinity
print(f"\nMissing values dalam matriks: {np.isnan(decision_matrix).sum()}")
print(f"Infinity values dalam matriks: {np.isinf(decision_matrix).sum()}")

# Sample matriks keputusan
print("\nSample matriks keputusan (5 buku pertama):")
sample_df = pd.DataFrame(decision_matrix[:5], columns=criteria_columns)
sample_df.index = df_topsis['JUDUL'].iloc[:5]
print(sample_df)

## 8. Run TOPSIS Algorithm

In [ ]:
# Jalankan algoritma TOPSIS
print("=== MENJALANKAN ALGORITMA TOPSIS ===")

# Inisialisasi dan jalankan TOPSIS
topsis = TOPSIS(decision_matrix, weights, criteria_types)
scores = topsis.run_topsis()

print("✓ TOPSIS berhasil dijalankan")
print(f"✓ Scores dihitung untuk {len(scores)} buku")

# Tambahkan skor ke dataframe
df_results = df_topsis.copy()
df_results['TOPSIS_SCORE'] = scores

# Urutkan berdasarkan skor tertinggi
df_results = df_results.sort_values('TOPSIS_SCORE', ascending=False).reset_index(drop=True)

print("\n=== TOP 10 BUKU DENGAN SKOR TERTINGGI ===")
top_10 = df_results.head(10)[['JUDUL', 'JUMLAH_USULAN', 'STOK_PENERBIT', 'HARGA', 'KEBARUAN', 'TOPSIS_SCORE']]
for i, row in top_10.iterrows():
    print(f"{i+1:2d}. {row['JUDUL'][:50]:<50} | Score: {row['TOPSIS_SCORE']:.4f}")

In [ ]:
# Tampilkan detail proses TOPSIS
print("=== DETAIL PROSES TOPSIS ===")

print("1. Normalized Matrix (5 alternatif pertama):")
normalized_sample = pd.DataFrame(topsis.normalized_matrix[:5], 
                                columns=criteria_columns,
                                index=df_results['JUDUL'].iloc[:5])
print(normalized_sample.round(4))

print("\n2. Weighted Normalized Matrix (5 alternatif pertama):")
weighted_sample = pd.DataFrame(topsis.weighted_matrix[:5], 
                              columns=criteria_columns,
                              index=df_results['JUDUL'].iloc[:5])
print(weighted_sample.round(4))

print("\n3. Ideal Solutions:")
ideal_df = pd.DataFrame({
    'Positive Ideal Solution (A+)': topsis.ideal_best,
    'Negative Ideal Solution (A-)': topsis.ideal_worst
}, index=criteria_columns)
print(ideal_df.round(4))

print("\n4. Distances (5 alternatif teratas):")
distance_df = pd.DataFrame({
    'Distance to A+': topsis.distances_best[:5],
    'Distance to A-': topsis.distances_worst[:5],
    'TOPSIS Score': scores[:5]
}, index=df_results['JUDUL'].iloc[:5])
print(distance_df.round(4))

## 9. Final Results - Top 5 Prioritized Books

In [ ]:
# HASIL AKHIR: 5 BUKU TERATAS YANG DIPRIORITASKAN
print("=" * 80)
print("🏆 HASIL AKHIR: 5 BUKU YANG PALING DIPRIORITASKAN UNTUK DIADAKAN")
print("=" * 80)

top_5_books = df_results.head(5)

for i, (idx, book) in enumerate(top_5_books.iterrows(), 1):
    print(f"\n📚 PERINGKAT {i}")
    print(f"Judul: {book['JUDUL']}")
    print(f"ISBN: {book['ISBN']}")
    print(f"TOPSIS Score: {book['TOPSIS_SCORE']:.4f}")
    print("-" * 50)
    print(f"📊 Detail Kriteria:")
    print(f"   • Jumlah Usulan: {book['JUMLAH_USULAN']} (Bobot: 35%)")
    print(f"   • Stok Penerbit: {book['STOK_PENERBIT']} (Bobot: 25%)")
    print(f"   • Harga: Rp {book['HARGA']:,.0f} (Bobot: 25%)")
    print(f"   • Tahun Terbit: {book['KEBARUAN']:.0f} (Bobot: 15%)")

print("\n" + "=" * 80)
print("📈 SUMMARY STATISTIK TOP 5:")
print("=" * 80)
summary_stats = top_5_books[['JUMLAH_USULAN', 'STOK_PENERBIT', 'HARGA', 'KEBARUAN', 'TOPSIS_SCORE']].describe()
print(summary_stats.round(2))

## 10. Visualization and Analysis

In [ ]:
# Visualisasi hasil TOPSIS
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Analisis Hasil TOPSIS: Top 5 Buku Prioritas', fontsize=16, fontweight='bold')

# 1. Bar chart TOPSIS scores
top_10_viz = df_results.head(10)
axes[0,0].barh(range(len(top_10_viz)), top_10_viz['TOPSIS_SCORE'], color='skyblue')
axes[0,0].set_yticks(range(len(top_10_viz)))
axes[0,0].set_yticklabels([title[:30] + '...' if len(title) > 30 else title 
                          for title in top_10_viz['JUDUL']], fontsize=8)
axes[0,0].set_xlabel('TOPSIS Score')
axes[0,0].set_title('Top 10 Buku berdasarkan TOPSIS Score')

# 2. Scatter plot: Harga vs Jumlah Usulan
top_20 = df_results.head(20)
scatter = axes[0,1].scatter(top_20['HARGA'], top_20['JUMLAH_USULAN'], 
                           c=top_20['TOPSIS_SCORE'], cmap='viridis', s=100, alpha=0.7)
axes[0,1].set_xlabel('Harga (Rupiah)')  
axes[0,1].set_ylabel('Jumlah Usulan')
axes[0,1].set_title('Harga vs Jumlah Usulan (Top 20)')
plt.colorbar(scatter, ax=axes[0,1], label='TOPSIS Score')

# 3. Distribution of criteria for top 5
top_5_criteria = top_5_books[criteria_columns]
axes[1,0].boxplot([top_5_criteria[col] for col in criteria_columns], 
                  labels=['Usulan', 'Stok', 'Harga', 'Tahun'])
axes[1,0].set_title('Distribusi Kriteria - Top 5 Buku')
axes[1,0].set_ylabel('Nilai')

# 4. Pie chart bobot kriteria
axes[1,1].pie(weights, labels=[col.replace('_', ' ') for col in criteria_columns], 
              autopct='%1.1f%%', startangle=90)
axes[1,1].set_title('Bobot Kriteria TOPSIS')

plt.tight_layout()
plt.show()

# Tabel perbandingan top 5
print("\n📊 TABEL PERBANDINGAN TOP 5 BUKU:")
display_cols = ['JUDUL', 'JUMLAH_USULAN', 'STOK_PENERBIT', 'HARGA', 'KEBARUAN', 'TOPSIS_SCORE']
comparison_table = top_5_books[display_cols].copy()
comparison_table['RANKING'] = range(1, 6)
comparison_table = comparison_table[['RANKING'] + display_cols]
print(comparison_table.to_string(index=False))

## 11. Conclusions and Recommendations

### Kesimpulan dan Rekomendasi

**Hasil Analisis TOPSIS:**

Berdasarkan analisis menggunakan metode TOPSIS (Technique for Order Preference by Similarity to Ideal Solution), telah berhasil ditentukan 5 buku yang paling diprioritaskan untuk diadakan atau ditambah stoknya di perpustakaan.

**Kriteria yang Digunakan:**
1. **Jumlah Usulan** (35% bobot) - Benefit: Semakin banyak usulan, semakin prioritas
2. **Stok Penerbit** (25% bobot) - Benefit: Semakin banyak stok tersedia, semakin mudah pengadaan
3. **Harga** (25% bobot) - Cost: Semakin murah, semakin baik untuk anggaran
4. **Kebaruan/Tahun Terbit** (15% bobot) - Benefit: Semakin baru, semakin relevan

**Keunggulan Metode TOPSIS:**
- Mempertimbangkan jarak ke solusi ideal positif dan negatif
- Memberikan ranking yang objektif berdasarkan multiple criteria
- Dapat menangani kriteria benefit dan cost secara bersamaan
- Hasil berupa skor 0-1 yang mudah diinterpretasi

**Rekomendasi:**
1. Prioritaskan pengadaan 5 buku teratas sesuai hasil analisis
2. Pertimbangkan anggaran yang tersedia dengan total estimasi biaya
3. Koordinasi dengan penerbit untuk memastikan ketersediaan stok
4. Monitor tingkat peminjaman setelah pengadaan untuk evaluasi

**Catatan Metodologi:**
- Data preprocessing dilakukan dengan menggabungkan 3 dataset terpisah
- Normalisasi menggunakan vector normalization
- Bobot kriteria dapat disesuaikan sesuai kebijakan perpustakaan